**Gensim是一个免费的 Python库，旨在自动从文档中提取语义主题，尽可能有效（计算机方式）和不费力（人性化）。  
Gensim旨在处理原始的非结构化数字文本（“ 纯文本 ”）。gensim中的算法如潜在语义分析，潜在狄利克雷分配和随机预测 通过检查训练文档语料库中单词的统计共现模式来发现文档的语义结构。这些算法是无监督的，这意味着不需要人工输入 - 你只需要一个纯文本文档的语料库。
一旦找到这些统计模式，任何纯文本文档都可以用新的语义表示简洁地表达，并且针对与其他文档的主题相似性进行查询。**  

**核心概念**  
整个gensim软件包围绕着语料库，矢量和 模型的概念。

**文集**  
数字文件的集合。该集合用于自动推断文档结构，主题等。因此，该集合也称为训练语料库。这个推断的潜在结构可以在以后用于将主题分配给新文档，这些文档没有出现在训练语料库中。不需要人工干预（如手动标记文档或创建其他元数据）。  
**向量**  
在矢量空间模型（VSM）中，每个文档都由一组特征表示。例如，一个特征可以被认为是一个问题 - 答案对：

单词splonge在文档中出现多少次？零。
文件包含多少段？二。
文档使用多少种字体？五。
问题通常只能由它的一个整数标识符（如所表示1，2和3在此），因此，该文件的表示变得一系列像对。如果我们事先知道所有问题，我们可以让它们隐含并且简单地写。这个答案序列可以被认为是一个向量（在这种情况下是一个三维向量）。出于实际的目的，只允许回答（或可以转换为）单个实数的问题。(1, 0.0), (2, 2.0), (3, 5.0)(0.0, 2.0, 5.0)

每个文件的问题都是一样的，所以通过查看两个向量（代表两个文件），我们希望能够得出如下结论：“这两个向量中的数字非常相似，因此原始文件必须相似也是“。当然，这样的结论是否符合现实取决于我们如何选择我们的问题。

**稀疏矢量**  
通常情况下，大多数问题的答案是0.0。为了节省空间，我们从文档的表示中省略它们，并且只写（注意缺失的）。由于所有问题的集合都是事先知道的，所以文档的稀疏表示中的所有缺失特征可以明确地解析为零。(2, 2.0), (3, 5.0)(1, 0.0)0.0

Gensim没有规定任何特定的语料库格式; 一个语料库就是当迭代时连续产生这些稀疏向量的东西。例如，set（（（（2，2.0），（3，5.0）），（（0,1.0），（3,1.0））））是两个文档的平凡语料库，每个文档都有两个非零特征-对组合。

**模型**  
我们使用模型作为抽象术语，指的是从一个文档表示到另一个文档表示的转换。在gensim中，文档被表示为向量，所以模型可以被认为是两个向量空间之间的转换。这个转变的细节是从训练语料库中学到的。

例如，考虑一种转换，它采用单词出现的原始计数并对它们进行加权，以便打折常用单词并提升罕见单词。任何特定单词加权的确切数量由训练语料库中该单词的相对频率决定。当我们应用这个模型时，我们从一个向量空间（包含原始字数）转换为另一个（包含加权计数）。

# 01语料库和矢量空间(Corpora_and_Vector _Spaces)

In [1]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)#记录日志事件

## 从字符串到向量（From Strings to Vectors）

In [2]:
from gensim import corpora
#几个不同的句子或者文档
documents = ["Human machine interface for lab abc computer applications",
              "A survey of user opinion of computer system response time",
              "The EPS user interface management system",
              "System and human system engineering testing of EPS",
              "Relation of user perceived response time to error measurement",
              "The generation of random binary unordered trees",
              "The intersection graph of paths in trees",
              "Graph minors IV Widths of trees and well quasi ordering",
              "Graph minors A survey"]

C:\Users\Administrator\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2018-03-21 10:30:12,002 : INFO : 'pattern' package not found; tag filters are not available for English


In [3]:
#进行简单的文本处理(停用词)
stoplist = set('for a of the and to in'.split())#停用词
texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]
print(texts);print()

#删除只出现一次的单词
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token  for token in text if frequency[token] > 1] for text in texts]
print(texts);print()

from pprint import pprint
pprint(texts)




[['human', 'machine', 'interface', 'lab', 'abc', 'computer', 'applications'], ['survey', 'user', 'opinion', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'management', 'system'], ['system', 'human', 'system', 'engineering', 'testing', 'eps'], ['relation', 'user', 'perceived', 'response', 'time', 'error', 'measurement'], ['generation', 'random', 'binary', 'unordered', 'trees'], ['intersection', 'graph', 'paths', 'trees'], ['graph', 'minors', 'iv', 'widths', 'trees', 'well', 'quasi', 'ordering'], ['graph', 'minors', 'survey']]

[['human', 'interface', 'computer'], ['survey', 'user', 'computer', 'system', 'response', 'time'], ['eps', 'user', 'interface', 'system'], ['system', 'human', 'system', 'eps'], ['user', 'response', 'time'], ['trees'], ['graph', 'trees'], ['graph', 'minors', 'trees'], ['graph', 'minors', 'survey']]

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 

In [4]:
#问题和ID之间的映射称为字典
dictionary = corpora.Dictionary(documents=texts)
dictionary.save('deerwester.dict') # store the dictionary, for future reference
print(dictionary);print()

#我们看到处理过的语料库中有十二个不同的单词，这意味着每个文档将由十二个数字表示（即，由一个12-D矢量）
print(dictionary.token2id)

#将标记化文档转换为矢量
new_doc = "Human computer interaction"
#该函数doc2bow()简单地计算每个不同单词的出现次数，将该单词转换为其整数单词id并将结果作为稀疏向量返回。
new_vec = dictionary.doc2bow(document=new_doc.lower().split())
print(new_vec)  # 单词 "interaction"没有出现过就被忽略了 


2018-03-21 10:30:12,177 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-21 10:30:12,181 : INFO : built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)
2018-03-21 10:30:12,184 : INFO : saving Dictionary object under deerwester.dict, separately None
2018-03-21 10:30:12,192 : INFO : saved deerwester.dict


Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)

{'computer': 0, 'human': 1, 'interface': 2, 'response': 3, 'survey': 4, 'system': 5, 'time': 6, 'user': 7, 'eps': 8, 'trees': 9, 'graph': 10, 'minors': 11}
[(0, 1), (1, 1)]


In [5]:
#处理好的语料生成
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('deerwester.mm', corpus) # store to disk, for later use
pprint(corpus)


2018-03-21 10:30:12,303 : INFO : storing corpus in Matrix Market format to deerwester.mm
2018-03-21 10:30:12,308 : INFO : saving sparse matrix to deerwester.mm
2018-03-21 10:30:12,311 : INFO : PROGRESS: saving document #0
2018-03-21 10:30:12,316 : INFO : saved 9x12 matrix, density=25.926% (28/108)
2018-03-21 10:30:12,319 : INFO : saving MmCorpus index to deerwester.mm.index


[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(2, 1), (5, 1), (7, 1), (8, 1)],
 [(1, 1), (5, 2), (8, 1)],
 [(3, 1), (6, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(4, 1), (10, 1), (11, 1)]]


## 语料库流 - 一次一个文件(Corpus Streaming – One Document at a Time)

In [6]:
#上面的小文本停留在内存(RAM)中，没问题，当文档很大时，就需要留处理了(前提是问题和ID之间的映射称为字典已经生成)
class Mycorpus(object):
    def __iter__(self):
        for line in open('mycorpus.txt'):
            yield dictionary.doc2bow(line.lower().split())
            
corpus_memory_friendly = Mycorpus()
print(corpus_memory_friendly)

for vector in corpus_memory_friendly:  # load one vector into memory at a time
    print(vector)


[(0, 1), (1, 1), (2, 1)]
[(0, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)]
[(2, 1), (5, 1), (7, 1), (8, 1)]
[(1, 1), (5, 2), (8, 1)]
[(3, 1), (6, 1), (7, 1)]
[(9, 1)]
[(9, 1), (10, 1)]
[(9, 1), (10, 1), (11, 1)]
[(4, 1), (10, 1), (11, 1)]


In [7]:
#构建字典而不将所有文本加载到内存中
from six import iteritems
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('mycorpus.txt'))
print(dictionary);print()

stop_ids = [dictionary.token2id[stopword] for stopword in stoplist if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

dictionary.filter_tokens(stop_ids + once_ids)# remove stop words and words that appear only once
dictionary.compactify()# remove gaps in id sequence after words that were removed
print(dictionary)

2018-03-21 10:30:12,534 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-21 10:30:12,535 : INFO : built Dictionary(42 unique tokens: ['abc', 'applications', 'computer', 'for', 'human']...) from 9 documents (total 69 corpus positions)


Dictionary(42 unique tokens: ['abc', 'applications', 'computer', 'for', 'human']...)

Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...)


## 语料库格式(Corpus Formats)

存在用于将矢量空间语料库（〜矢量序列）序列化到磁盘的多种文件格式。 Gensim通过前面提到的流式语料库接口实现它们：文档以懒惰的方式从一个文件读取（或存储到），一次一个文档，而不会将整个语料库一次读入主存储器。

In [8]:
# create a toy corpus of 2 documents, as a plain Python list
corpus = [[(1, 0.5)], []]  # make one document empty, for the heck of it
corpora.MmCorpus.serialize('corpus.mm', corpus) #市场矩阵格式(Market Matrix format)


2018-03-21 10:30:12,739 : INFO : storing corpus in Matrix Market format to corpus.mm
2018-03-21 10:30:12,742 : INFO : saving sparse matrix to corpus.mm
2018-03-21 10:30:12,743 : INFO : PROGRESS: saving document #0
2018-03-21 10:30:12,746 : INFO : saved 2x2 matrix, density=25.000% (1/4)
2018-03-21 10:30:12,749 : INFO : saving MmCorpus index to corpus.mm.index


In [9]:
#其他格式包括Joachim的SVMlight格式， Blei的LDA-C格式和 GibbsLDA ++格式
# Joachim’s SVMlight format, Blei’s LDA-C format and GibbsLDA++ format.
corpora.SvmLightCorpus.serialize('corpus.svmlight', corpus)
corpora.BleiCorpus.serialize('corpus.lda-c', corpus)
corpora.LowCorpus.serialize('corpus.low', corpus)

2018-03-21 10:30:12,872 : INFO : converting corpus to SVMlight format: corpus.svmlight
2018-03-21 10:30:12,874 : INFO : saving SvmLightCorpus index to corpus.svmlight.index
2018-03-21 10:30:12,877 : INFO : no word id mapping provided; initializing from corpus
2018-03-21 10:30:12,878 : INFO : storing corpus in Blei's LDA-C format into corpus.lda-c
2018-03-21 10:30:12,881 : INFO : saving vocabulary of 2 words to corpus.lda-c.vocab
2018-03-21 10:30:12,883 : INFO : saving BleiCorpus index to corpus.lda-c.index
2018-03-21 10:30:12,887 : INFO : no word id mapping provided; initializing from corpus
2018-03-21 10:30:12,889 : INFO : storing corpus in List-Of-Words format into corpus.low
2018-03-21 10:30:12,895 : WARNING : List-of-words format can only save vectors with integer elements; 1 float entries were truncated to integer value
2018-03-21 10:30:12,898 : INFO : saving LowCorpus index to corpus.low.index


In [10]:
#相反，要从Matrix Market文件加载语料库迭代器：
corpus = corpora.MmCorpus('corpus.mm')
#语料库对象是流，因此通常您将无法直接打印它们：
print(corpus)
#相反，要查看语料库的内容：
print(list(corpus))
for doc in corpus:
     print(doc)
#再转换成其他格式
corpora.BleiCorpus.serialize('corpus.lda-c', corpus)

#通过这种方式，gensim也可以用作高效的I / O格式转换工具：只需使用一种格式加载文档流，并立即以另一种格式保存

2018-03-21 10:30:13,176 : INFO : loaded corpus index from corpus.mm.index
2018-03-21 10:30:13,179 : INFO : initializing cython corpus reader from corpus.mm
2018-03-21 10:30:13,183 : INFO : accepted corpus with 2 documents, 2 features, 1 non-zero entries
2018-03-21 10:30:13,188 : INFO : no word id mapping provided; initializing from corpus
2018-03-21 10:30:13,190 : INFO : storing corpus in Blei's LDA-C format into corpus.lda-c
2018-03-21 10:30:13,194 : INFO : saving vocabulary of 2 words to corpus.lda-c.vocab
2018-03-21 10:30:13,198 : INFO : saving BleiCorpus index to corpus.lda-c.index


MmCorpus(2 documents, 2 features, 1 non-zero entries)
[[(1, 0.5)], []]
[(1, 0.5)]
[]


## 与NumPy和SciPy兼容

In [11]:
import gensim
import numpy as np
numpy_matrix = np.random.randint(10, size=[5,2])  # random matrix as an example
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
numpy_matrix = gensim.matutils.corpus2dense(corpus, num_terms=number_of_corpus_features)


NameError: name 'number_of_corpus_features' is not defined

In [12]:
import scipy.sparse
scipy_sparse_matrix = scipy.sparse.random(5,2)  # random sparse matrix as example
corpus = gensim.matutils.Sparse2Corpus(scipy_sparse_matrix)
scipy_csc_matrix = gensim.matutils.corpus2csc(corpus)


# 02主题和转换(Topics and Transformations)

## 转换接口

In [16]:
from gensim import corpora, models, similarities
import os
if(os.path.exists('deerwester.dict')):
    dictionary = corpora.Dictionary.load('deerwester.dict')
    corpus = corpora.MmCorpus('deerwester.mm')
    print("Used files generated from first tutorial")
else:
    print("Please run first tutorial to generate data set")
    

2018-03-21 10:42:51,173 : INFO : loading Dictionary object from deerwester.dict
2018-03-21 10:42:51,176 : INFO : loaded deerwester.dict
2018-03-21 10:42:51,179 : INFO : loaded corpus index from deerwester.mm.index
2018-03-21 10:42:51,181 : INFO : initializing cython corpus reader from deerwester.mm
2018-03-21 10:42:51,185 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


Used files generated from first tutorial


在本教程中，我将演示如何将文档从一个矢量表示转换为另一个矢量表示。这个过程有两个目标：

为了带出语料库中隐藏的结构，发现词之间的关系，并用它们以新的（有希望的）更有意义的方式描述文档。

使文档表示更加紧凑。这既提高了效率（新的表示消耗更少的资源）和有效性（边际数据趋势被忽略，降低噪音）。

## 创建转换

**转换总是在两个特定的向量空间之间转换。必须使用相同的向量空间（=相同的特征ID集合）进行训练以及后续的向量变换。如果不能使用相同的输入特征空间，例如应用不同的字符串预处理，使用不同的特征标识，或者在预期TfIdf矢量的情况下使用袋字输入向量，将导致转换调用期间的功能不匹配，从而导致垃圾输出和/或运行时异常。**

In [17]:
#转换是标准的Python对象，通常通过训练语料库进行初始化：
tfidf = models.TfidfModel(corpus=corpus)# step 1 -- initialize a model


2018-03-21 10:46:13,713 : INFO : collecting document frequencies
2018-03-21 10:46:13,715 : INFO : PROGRESS: processing document #0
2018-03-21 10:46:13,717 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)


## 转换矢量

In [23]:
#从现在开始，tfidf被视为只读对象，可用于将任何向量从旧的表示（bag-of-words整数计数）转换为新的表示（TfIdf实值权重）：
doc_bow = [(0, 1), (1, 1)]
print(tfidf[doc_bow]) # step 2 -- use the model to transform vectors

#或者对整个语料库应用转换：
#(调用model[corpus]仅在旧corpus 文档流中创建一个包装- 实际转换在文档迭代期间即时完成。)
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)


[(0, 0.7071067811865476), (1, 0.7071067811865476)]
[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(0, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.3244870206138555), (6, 0.44424552527467476), (7, 0.3244870206138555)]
[(2, 0.5710059809418182), (5, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(1, 0.49182558987264147), (5, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (6, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(4, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [25]:
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=2) # initialize an LSI transformation
corpus_lsi = lsi[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
lsi.print_topics(2)

2018-03-21 10:56:30,673 : INFO : using serial LSI version on this node
2018-03-21 10:56:30,674 : INFO : updating model with new documents
2018-03-21 10:56:30,676 : INFO : preparing a new chunk of documents
2018-03-21 10:56:30,677 : INFO : using 100 extra samples and 2 power iterations
2018-03-21 10:56:30,678 : INFO : 1st phase: constructing (12, 102) action matrix
2018-03-21 10:56:30,680 : INFO : orthonormalizing (12, 102) action matrix
2018-03-21 10:56:30,682 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-03-21 10:56:30,683 : INFO : computing the final decomposition
2018-03-21 10:56:30,685 : INFO : keeping 2 factors (discarding 47.565% of energy spectrum)
2018-03-21 10:56:30,686 : INFO : processed documents up to #9
2018-03-21 10:56:30,687 : INFO : topic #0(1.594): 0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"
2018-03-21 10:56:30,689 : INFO : topic #

[(0,
  '0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"time" + 0.060*"response" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"'),
 (1,
  '-0.460*"system" + -0.373*"user" + -0.332*"eps" + -0.328*"interface" + -0.320*"time" + -0.320*"response" + -0.293*"computer" + -0.280*"human" + -0.171*"survey" + 0.161*"trees"')]

In [27]:
#根据LSI的说法，“树”，“图”和“选修”都是相关词（对第一个主题的方向贡献最大），而第二个主题实际上与所有其他词有关。正如预期的那样，前五个文件与第二个主题更密切相关，而剩下的四个文件涉及第一个主题：
for doc in corpus_lsi:
    print(doc)

[(0, -0.066), (1, 0.520)] # "Human machine interface for lab abc computer applications"
[(0, -0.197), (1, 0.761)] # "A survey of user opinion of computer system response time"
[(0, -0.090), (1, 0.724)] # "The EPS user interface management system"
[(0, -0.076), (1, 0.632)] # "System and human system engineering testing of EPS"
[(0, -0.102), (1, 0.574)] # "Relation of user perceived response time to error measurement"
[(0, -0.703), (1, -0.161)] # "The generation of random binary unordered trees"
[(0, -0.877), (1, -0.168)] # "The intersection graph of paths in trees"
[(0, -0.910), (1, -0.141)] # "Graph minors IV Widths of trees and well quasi ordering"
[(0, -0.617), (1, 0.054)] # "Graph minors A survey"


#通过save和load进行持久化操作
lsi.save('model.lsi')# same for tfidf, lda, ...
lsi = models.LsiModel.load('model.lsi')


2018-03-21 11:03:48,405 : INFO : saving Projection object under model.lsi.projection, separately None
2018-03-21 11:03:48,409 : INFO : saved model.lsi.projection
2018-03-21 11:03:48,410 : INFO : saving LsiModel object under model.lsi, separately None
2018-03-21 11:03:48,412 : INFO : not storing attribute projection
2018-03-21 11:03:48,413 : INFO : not storing attribute dispatcher
2018-03-21 11:03:48,416 : INFO : saved model.lsi
2018-03-21 11:03:48,418 : INFO : loading LsiModel object from model.lsi
2018-03-21 11:03:48,421 : INFO : loading id2word recursively from model.lsi.id2word.* with mmap=None
2018-03-21 11:03:48,424 : INFO : setting ignored attribute projection to None
2018-03-21 11:03:48,426 : INFO : setting ignored attribute dispatcher to None
2018-03-21 11:03:48,428 : INFO : loaded model.lsi
2018-03-21 11:03:48,431 : INFO : loading LsiModel object from model.lsi.projection
2018-03-21 11:03:48,435 : INFO : loaded model.lsi.projection


[(0, 0.06600783396090572), (1, -0.5200703306361847)]
[(0, 0.19667592859142874), (1, -0.7609563167700035)]
[(0, 0.08992639972446731), (1, -0.7241860626752509)]
[(0, 0.07585847652178428), (1, -0.632055158600343)]
[(0, 0.10150299184980405), (1, -0.5737308483002947)]
[(0, 0.7032108939378298), (1, 0.1611518021402609)]
[(0, 0.8774787673119819), (1, 0.1675890686465978)]
[(0, 0.9098624686818566), (1, 0.140865536287194)]
[(0, 0.6165825350569283), (1, -0.053929075663891005)]


## 可用的转换

In [35]:
#Gensim实现了几种流行的矢量空间模型算法：
'''术语频率*反向文档频率，Tf-Idf 在初始化期间需要一个词袋（整数值）训练语料库。
在转换过程中，它将采用一个向量并返回具有相同维度的另一个向量，除了训练语料库中罕见
的特征将增加其值。因此它将整数值向量转换为实值向量，同时保持维数不变。它也可以选择
将结果向量归一化为（欧几里得）单位长度。'''
model = models.TfidfModel(corpus, normalize=True)

'''潜在语义索引LSI（有时候称为LSA） 将文档从文字袋或TfIdf加权空间转换为低维潜在空间。
对于上述玩具语料库，我们只使用了2个潜在维度，但在真实语料库中，目标维度为200-500
被推荐为“黄金标准”'''
model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)

'''LSI培训的独特之处在于我们可以在任何时候继续“培训”，只需提供更多培训文档即可。这
是通过对基础模型的增量更新完成的，这个过程称为在线培训。由于这一特性，输入文档流甚至
可能是无限的 - 只是在LSI新文档到达时继续提供，同时将计算出的转换模型作为只读使用！'''
# model.add_documents(another_tfidf_corpus)# now LSI has been trained on tfidf_corpus + another_tfidf_corpus
# lsi_vec = model[tfidf_vec] # convert some new document into the LSI space, without affecting the model
# model.add_documents(more_documents) # tfidf_corpus + another_tfidf_corpus + more_documents
# lsi_vec = model[tfidf_vec]

#gensim使用了一种新的在线增量流式分布式训练算法

'''随机投影，RP旨在减少向量空间维度。这是一种非常有效的（内存和CPU友好型）方法，通过抛
出一点随机性来逼近文档之间的TfIdf距离。根据您的数据集，建议的目标维度也是数百/数千。'''
model = models.RpModel(corpus_tfidf, num_topics=500)

'''潜在Dirichlet分配，LDA 是从包含词计数到低维度主题空间的又一转变。LDA是LSA（也称为多项
PCA）的概率扩展，因此LDA的主题可以解释为概率分布。这些分布与LSA一样，是从训练语料库中自动
推断出来的。文档又被解释为这些主题的（软）混合体（再次，就像LSA一样）。'''
model = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

'''分层Dirichlet过程，HDP 是一种非参数贝叶斯方法（请注意缺少的请求主题数量）'''
model = models.HdpModel(corpus, id2word=dictionary)

#值得重申的是，这些都是独一无二的，增量式的实现，它们不需要一次将所有训练语料库存在主内存中。

2018-03-21 23:19:44,421 : INFO : collecting document frequencies
2018-03-21 23:19:44,434 : INFO : PROGRESS: processing document #0
2018-03-21 23:19:44,435 : INFO : calculating IDF weights for 9 documents and 11 features (28 matrix non-zeros)
2018-03-21 23:19:44,436 : INFO : using serial LSI version on this node
2018-03-21 23:19:44,437 : INFO : updating model with new documents
2018-03-21 23:19:44,439 : INFO : preparing a new chunk of documents
2018-03-21 23:19:44,440 : INFO : using 100 extra samples and 2 power iterations
2018-03-21 23:19:44,441 : INFO : 1st phase: constructing (12, 400) action matrix
2018-03-21 23:19:44,444 : INFO : orthonormalizing (12, 400) action matrix
2018-03-21 23:19:44,446 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-03-21 23:19:44,448 : INFO : computing the final decomposition
2018-03-21 23:19:44,453 : INFO : keeping 9 factors (discarding 0.000% of energy spectrum)
2018-03-21 23:19:44,456 : INFO : processed documents up to #9
2018-03-21 23:19:4

# 03相似性查询

## 相似的接口（Similarity interface）

在前面有关语料库和矢量空间以及主题和变换的教程中，我们介绍了如何在矢量空间模型中创建语料库以及如何在不同的矢量空间之间对其进行变换。这种讨论的一个常见原因是我们想要确定 文档对之间的相似性，或者特定文档与其他文档集（如用户查询与索引文档）之间的相似度。

In [38]:
from gensim import corpora, models, similarities
dictionary = corpora.Dictionary.load('deerwester.dict')
corpus = corpora.MmCorpus('deerwester.mm')# comes from the first tutorial, "From strings to vectors"
print(corpus)


2018-03-22 22:00:43,886 : INFO : loading Dictionary object from deerwester.dict
2018-03-22 22:00:43,910 : INFO : loaded deerwester.dict
2018-03-22 22:00:43,913 : INFO : loaded corpus index from deerwester.mm.index
2018-03-22 22:00:43,914 : INFO : initializing cython corpus reader from deerwester.mm
2018-03-22 22:00:43,916 : INFO : accepted corpus with 9 documents, 12 features, 28 non-zero entries


MmCorpus(9 documents, 12 features, 28 non-zero entries)


In [39]:
#按照Deerwester的例子，我们首先使用这个小型语料库来定义一个2维LSI空间：
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)


2018-03-22 22:01:23,293 : INFO : using serial LSI version on this node
2018-03-22 22:01:23,404 : INFO : updating model with new documents
2018-03-22 22:01:23,748 : INFO : preparing a new chunk of documents
2018-03-22 22:01:24,078 : INFO : using 100 extra samples and 2 power iterations
2018-03-22 22:01:24,088 : INFO : 1st phase: constructing (12, 102) action matrix
2018-03-22 22:01:24,294 : INFO : orthonormalizing (12, 102) action matrix
2018-03-22 22:01:24,915 : INFO : 2nd phase: running dense svd on (12, 9) matrix
2018-03-22 22:01:25,226 : INFO : computing the final decomposition
2018-03-22 22:01:25,278 : INFO : keeping 2 factors (discarding 43.156% of energy spectrum)
2018-03-22 22:01:25,380 : INFO : processed documents up to #9
2018-03-22 22:01:25,412 : INFO : topic #0(3.341): 0.644*"system" + 0.404*"user" + 0.301*"eps" + 0.265*"time" + 0.265*"response" + 0.240*"computer" + 0.221*"human" + 0.206*"survey" + 0.198*"interface" + 0.036*"graph"
2018-03-22 22:01:25,414 : INFO : topic #1(2

In [40]:
'''现在假设用户在查询“人机交互”中键入。我们希望按照与此查询相关的降序对我们的九个语料
库文档进行排序。与现代搜索引擎不同，这里我们只关注可能相似点的一个方面 - 关于他们文本
（词）的明显的语义相关性。没有超链接，没有随机游走静态等级，只是布尔关键字匹配的语义扩
展：'''
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]# convert the query to LSI space
print(vec_lsi)


[(0, 0.4618210045327158), (1, 0.07002766527899963)]


In [41]:
'''另外，我们将考虑余弦相似性 来确定两个向量的相似性。余弦相似性是向量空间建模中的
一种标准度量，但向量表示概率分布的任何地方， 不同的相似性度量 可能更合适。'''
index = similarities.MatrixSimilarity(lsi[corpus])# transform corpus to LSI space and index it
index

'''similarities.MatrixSimilarity只有当整组矢量适合内存时，该类才适用。例如，当使用这个类时，
一百万份文档的语料库需要在256维LSI空间中使用2GB的RAM。如果没有2GB的可用RAM，则需要使用
similarities.Similarity该类。此类在固定内存中运行，方法是将索引拆分到磁盘上的多个文件中，
称为碎片。它采用similarities.MatrixSimilarity和similarities.SparseMatrixSimilarity内部，
所以还是快，虽然稍微复杂一些。'''



2018-03-22 22:05:55,002 : WARNING : scanning corpus to determine the number of features (consider setting `num_features` explicitly)
2018-03-22 22:05:55,225 : INFO : creating matrix with 9 documents and 2 features


In [42]:
#索引持久性通过标准save()和load()函数处理：
index.save('deerwester.index')
index = similarities.MatrixSimilarity.load('deerwester.index')

'''这是所有类似的索引类真（similarities.Similarity， similarities.MatrixSimilarity
和similarities.SparseMatrixSimilarity）。同样在下面，索引可以是任何这些的一个对象。
如果有疑问，请使用similarities.Similarity它，因为它是最具可扩展性的版本，并且它还
支持稍后向索引添加更多文档。'''

2018-03-22 22:08:45,420 : INFO : saving MatrixSimilarity object under deerwester.index, separately None
2018-03-22 22:08:45,634 : INFO : saved deerwester.index
2018-03-22 22:08:45,636 : INFO : loading MatrixSimilarity object from deerwester.index
2018-03-22 22:08:45,638 : INFO : loaded deerwester.index


## 执行查询

In [47]:
#为了获得我们的查询文档与九个索引文档的相似性：
sims = index[vec_lsi]# perform a similarity query against the corpus
print(list(enumerate(sims))) # print (document_number, document_similarity) 2-tuples

#余弦度量在<-1,1>范围内返回相似性（越大越相似），因此第一个文档的得分为0.99809301等。

[(0, 0.998093), (1, 0.93748635), (2, 0.9984453), (3, 0.9865886), (4, 0.90755945), (5, -0.12416792), (6, -0.10639259), (7, -0.09879464), (8, 0.050041765)]


In [48]:
#使用一些标准的Python魔法，我们按照降序对这些相似性进行排序，并获得查询“人机交互”的最终答案：
sims = sorted(enumerate(sims), key=lambda item: -item[1])
print(sims) # print sorted (document number, similarity score) 2-tuples

# [(2, 0.99844527), # The EPS user interface management system
# (0, 0.99809301), # Human machine interface for lab abc computer applications
# (3, 0.9865886), # System and human system engineering testing of EPS
# (1, 0.93748635), # A survey of user opinion of computer system response time
# (4, 0.90755945), # Relation of user perceived response time to error measurement
# (8, 0.050041795), # Graph minors A survey
# (7, -0.098794639), # Graph minors IV Widths of trees and well quasi ordering
# (6, -0.1063926), # The intersection graph of paths in trees
# (5, -0.12416792)] # The generation of random binary unordered trees


[(2, 0.9984453), (0, 0.998093), (3, 0.9865886), (1, 0.93748635), (4, 0.90755945), (8, 0.050041765), (7, -0.09879464), (6, -0.10639259), (5, -0.12416792)]


# 英语维基百科上的实验(Experiments on the English Wikipedia)

## 准备语料库

首先，从http://download.wikimedia.org/enwiki/  下载所有维基百科文章的转储 （您需要文件enwiki-latest-pages-articles.xml.bz2或enwiki-YYYYMMDD-pages-articles.xml。 bz2用于日期特定的转储）。该文件大小约为8GB，包含英文维基百科全部文章（的压缩版本）。

将文章转换为纯文本（处理Wiki标记）并将结果存储为稀疏TF-IDF向量。在Python中，这很容易即时执行，我们甚至不需要将整个存档解压缩到磁盘。gensim中包含一个脚本 就是这样做的，运行：

$ python -m gensim.scripts.make_wiki

这个预处理步骤在8.2GB压缩维基转储（一个提取词典，一个创建和存储稀疏向量）上进行两次传递，并且在我的笔记本上花费大约9个小时，因此您可能想要喝杯咖啡或二。

另外，您需要大约35GB的可用磁盘空间来存储稀疏输出向量。我建议立即压缩这些文件，例如用bzip2（低至〜13GB）。Gensim可以直接处理压缩文件，因此可以节省磁盘空间。


## 潜在语义分析

In [ ]:
#数据量太大，弄不了了
#首先让我们加载在上面第二步中创建的语料库迭代器和词典
import logging, gensim
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## load id->word mapping (the dictionary), one of the results of step 2 above
id2word = gensim.corpora.Dictionary.load_from_text(wiki_en_wordids.txt)
# load corpus iterator
mm = gensim.corpora.MmCorpus('wiki_en_tfidf.mm')
# mm = gensim.corpora.MmCorpus('wiki_en_tfidf.mm.bz2') # use this if you compressed the TFIDF output (recommended)
print(mm)


In [ ]:
#现在我们准备计算英语维基百科的LSA：
## extract 400 LSI topics; use the default one-pass algorithm
lsi = gensim.models.lsimodel.LsiModel(corpus=mm, id2word=id2word, num_topics=400)
# print the most contributing words (both positively and negatively) for each of the first ten topics
lsi.print_topics(10)


In [ ]:
'''gensim中使用的算法只需要查看每个输入文档一次，因此它适用于文档以不可重复流的形式出现
的环境，或适用于多次存储/迭代整个语料库的成本过高的环境。'''

In [ ]:
#潜在Dirichlet分配
import logging, gensim
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# load id->word mapping (the dictionary), one of the results of step 2 above
id2word = gensim.corpora.Dictionary.load_from_text('wiki_en_wordids.txt')
# load corpus iterator
mm = gensim.corpora.MmCorpus('wiki_en_tfidf.mm')
# mm = gensim.corpora.MmCorpus('wiki_en_tfidf.mm.bz2') # use this if you compressed the TFIDF output
print(mm)


In [ ]:
'''我们将运行在线LDA（参见Hoffman等人[3]），这是一种接受大量文档，更新LDA模型，接受另一
个大块，更新模型等的算法。在线LDA可以与处理整个语料库（一次全通）的批处理LDA形成对照，
然后更新模型，然后是另一个通过，另一个更新......不同之处在于，给定一个合理固定的文档流
（没有太多的主题漂移），小块（subcorpora）上的在线更新本身非常好，因此模型估计收敛得更快
。因此，我们可能只需要对整个语料库进行一次完整的传递：如果语料库有300万篇文章，并且在每
10,000篇文章之后更新一次，这意味着我们将一次完成300次更新，很可能足以有一个非常准确的话
题估计：'''
## extract 100 LDA topics, using 1 pass and updating once every 1 chunk (10,000 documents)
lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=100, update_every=1, chunksize=10000, passes=1)
# print the most contributing words for 20 randomly selected topics
lda.print_topics(20)

'''注意LDA和LSA运行的两个区别：我们要求LSA提取400个主题，LDA只有100个主题（所以速度差异
实际上甚至更大）。其次，gensim中的LSA实现是真正在线的：如果输入流的性质随时间变化，LSA将
重新定位自己以反映这些变化，并进行相当少量的更新。相比之下，LDA并不是真正的在线（ 尽管文
献[3]的名称），因为后期更新对模型的影响逐渐减弱。如果输入文档流中存在主题漂移，则LDA会感
到困惑，并且越来越慢地适应新的事态。

简而言之，如果使用LDA随着时间的推移逐渐向模型添加新文档，请小心。LDA的批量使用，其中整个
训练语料库事先已知或未显示主题漂移，可以，并且不受影响。'''

In [ ]:
#要运行批次LDA（不在线），请使用以下方式培训LdaModel：
# extract 100 LDA topics, using 20 full passes, no online updates
lda = gensim.models.ldamodel.LdaModel(corpus=mm, id2word=id2word, num_topics=100, update_every=0, passes=20)


In [ ]:
#像往常一样，一个训练有素的模型可以用来将新的，看不见的文档（简单的词袋计数向量）转换为LDA主题分布
doc_lda = lda[doc_bow]


In [50]:
from gensim.models import Word2Vec
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]

model = Word2Vec(sentences, min_count=1)
say_vector = model['say']  # get vector for word
say_vector

2018-03-25 10:04:06,620 : INFO : collecting all words and their counts
2018-03-25 10:04:06,622 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-25 10:04:06,623 : INFO : collected 5 word types from a corpus of 6 raw words and 2 sentences
2018-03-25 10:04:06,626 : INFO : Loading a fresh vocabulary
2018-03-25 10:04:06,627 : INFO : min_count=1 retains 5 unique words (100% of original 5, drops 0)
2018-03-25 10:04:06,628 : INFO : min_count=1 leaves 6 word corpus (100% of original 6, drops 0)
2018-03-25 10:04:06,629 : INFO : deleting the raw counts dictionary of 5 items
2018-03-25 10:04:06,630 : INFO : sample=0.001 downsamples 5 most-common words
2018-03-25 10:04:06,631 : INFO : downsampling leaves estimated 0 word corpus (7.5% of prior 6)
2018-03-25 10:04:06,632 : INFO : estimated required memory for 5 words and 100 dimensions: 6500 bytes
2018-03-25 10:04:06,633 : INFO : resetting layer weights
2018-03-25 10:04:06,646 : INFO : training model with 3 workers o

array([ 3.6672794e-03,  2.3785748e-03, -1.2979079e-03, -2.2986489e-03,
        3.8589868e-03,  4.7677383e-03, -8.4622815e-04, -4.7469721e-03,
        4.5546829e-03, -2.1930039e-03,  7.8014639e-04, -1.7201888e-03,
       -2.5572879e-03, -3.7275692e-03, -1.2652479e-03, -2.4822517e-03,
       -3.3013052e-03, -3.9827703e-03,  7.9437724e-04, -2.5375409e-03,
        3.2035625e-03,  4.6119727e-03, -4.8709619e-03, -4.3224655e-03,
        3.8996288e-03, -7.2479033e-04, -1.1487225e-03,  3.0452790e-04,
       -3.0824414e-03, -1.0298850e-03,  3.1206936e-03,  4.0110103e-03,
       -2.7508582e-03, -3.9044614e-03, -2.9462778e-03, -2.9102932e-03,
       -2.8171660e-03,  2.9010044e-03, -2.3535555e-04,  5.0747598e-04,
        2.3167240e-03,  2.7977065e-03, -4.0146853e-03,  1.2916119e-03,
        1.6181866e-03,  2.6329991e-03,  7.3889247e-04,  2.7017347e-03,
       -2.2440269e-03, -8.5438509e-04, -1.2963975e-03,  3.9401818e-03,
        8.8969828e-04,  2.4732088e-03,  3.1242839e-03,  3.3276102e-03,
      

In [51]:
say_vector.shape

(100,)

In [60]:
temp = [model[word] for word in sentences]
len(temp[0])

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


3

In [61]:
from gensim.models import FastText
sentences = [["cat", "say", "meow"], ["dog", "say", "woof"]]

model = FastText(sentences, min_count=1)
say_vector = model['say']  # get vector for word
of_vector = model['of']  # get vector for out-of-vocab word
print(say_vector)
print(of_vector)

2018-03-25 10:19:46,263 : INFO : collecting all words and their counts
2018-03-25 10:19:46,264 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-03-25 10:19:46,268 : INFO : collected 5 word types from a corpus of 6 raw words and 2 sentences
2018-03-25 10:19:46,271 : INFO : Loading a fresh vocabulary
2018-03-25 10:19:46,273 : INFO : min_count=1 retains 5 unique words (100% of original 5, drops 0)
2018-03-25 10:19:46,274 : INFO : min_count=1 leaves 6 word corpus (100% of original 6, drops 0)
2018-03-25 10:19:46,276 : INFO : deleting the raw counts dictionary of 5 items
2018-03-25 10:19:46,278 : INFO : sample=0.001 downsamples 5 most-common words
2018-03-25 10:19:46,279 : INFO : downsampling leaves estimated 0 word corpus (7.5% of prior 6)
2018-03-25 10:19:46,359 : INFO : estimated required memory for 5 words, 38 buckets and 100 dimensions: 22244 bytes
2018-03-25 10:19:46,400 : INFO : resetting layer weights
2018-03-25 10:19:46,829 : INFO : Total number of ng

[-2.48374348e-03  1.28183200e-03  2.78071064e-04  1.17748801e-03
 -2.84763589e-03  9.77993477e-04  7.77854701e-04  3.20158107e-03
 -3.23760835e-03  7.47663376e-04 -2.53082108e-04 -1.91538595e-03
  4.54709720e-04 -5.62755333e-04 -3.87221808e-03  1.28175726e-03
 -1.76023715e-03  7.72499421e-04  1.95931108e-03 -2.86182249e-03
 -4.55376838e-04  3.05658067e-03  2.28731777e-03  8.24262388e-05
 -1.02034304e-03  3.90086439e-03 -2.50886264e-03 -1.91701506e-03
 -2.79970840e-03 -6.84790313e-04 -3.51232733e-03  7.81246563e-05
 -1.41762511e-03  3.18209408e-03  1.07611215e-03 -3.61893000e-03
  1.30862638e-03  1.16036169e-03 -3.37773818e-03  4.24797880e-03
 -1.71071826e-03  6.81362930e-04 -2.27297656e-03  2.86741694e-03
 -1.21385325e-04  5.62858162e-03  2.79778475e-03  1.07019977e-03
 -1.81176583e-03 -1.60234573e-03 -2.64236447e-03  2.93838070e-03
 -1.13120431e-03 -1.86256308e-03  7.85981014e-04 -1.54724543e-03
  1.97424321e-03 -3.16070416e-03  2.61859177e-03 -2.08307451e-04
 -1.47058594e-03  2.41366

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel\__main__.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
